In [1]:
import os
from shutil import copyfile
from tqdm import tqdm
import numpy as np
from tfrecord import TFRecordCreator
from hparams import hparams

In [2]:
tacotron_out = '/NVME_DISK/Tacotron-2/tacotron_output/gta/'
tacotron_dir = '/NVME_DISK/Tacotron-2/'

with open(os.path.join(tacotron_out, 'map.txt'), 'rt', encoding='utf-8') as f:
    tacotron_meta = f.read().strip().split('\n')
    tacotron_meta = [m.split('|') for m in tacotron_meta]

In [3]:
with open('training_data/train.txt', 'wt', encoding='utf-8') as f:
    for m in tqdm(tacotron_meta):
        audio_basename = os.path.basename(m[0])
        audio_dst = os.path.join('training_data/audios', audio_basename)
        copyfile(os.path.join(tacotron_dir, m[0]) , audio_dst)
        mel_basename = os.path.basename(m[2])
        mel_dst = os.path.join('training_data/mels', mel_basename)
        mel = np.load(os.path.join(tacotron_dir, m[2]))
        mel = np.clip(mel, -4, 4)
        mel = ((mel + 4) / 8).astype(np.float32)
        np.save(mel_dst, mel, allow_pickle=False)
        row = f'{audio_basename}|{mel_basename}|-1|0|{m[4]}\n'
        f.write(row)
    f.flush()

100%|██████████| 11036/11036 [00:12<00:00, 893.47it/s]


In [4]:
creator = TFRecordCreator('training_data/train.txt', hparams)
creator.create_tfrecords()

100%|██████████| 10/10 [00:00<00:00, 19.18it/s]
